# Pandas Library

In [2]:
# import pandas as pd
import pandas as pd

## Importing Flat Files
- Such as csv and txt files with rows and cols
- `dataframe = pd.read_csv(filename[, sep][, comment][, na_values][, nrows][, header])`
    - `sep` is pandas version of delimiter with default `','`
    - `comment` takes the char that comments appear after (for python it's '#')
    - `na_values` takes a list of strings to identify as NA or NaN
    - `nrows` specifies an integer for the number of rows to retrieve
    - `header=None` if no header
    - view this dataframe with `.head()` method `dataframe.head()`

## Convert Dataframe to Numpy Array
- `array = dataframe.values`
    - these "values" must all be the same type